In [22]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
matplotlib.use('agg')
import pylab as plt
import pandas as pd
from keras.utils.np_utils import to_categorical

from keras.utils import np_utils
from sklearn import preprocessing as pp

import h5py
ss = pp.StandardScaler()

X_train = np.load("20170827_mawilab_flow_000flags_nosvc_features.pkl")
Y_train = np.load("20170827_mawilab_flow_000flags_nosvc_labels.pkl")


# Test Set 1 : 20170827_mawilab_flow_001
X_test1 = np.load("20170827_mawilab_flow_001flags_nosvc_features.pkl")
Y_test1 = np.load("20170827_mawilab_flow_001flags_nosvc_labels.pkl")
X_train = ss.fit_transform(X_train)
X_test1 = ss.fit_transform(X_test1)

# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test1 = np.reshape(X_test1, (X_test1.shape[0],X_test1.shape[1],1))

print(X_train.shape)
print(Y_train.shape)

(38361, 15, 1)
(38361, 2)


In [23]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(15, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(2),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 15, 64)            256       
_________________________________________________________________
activation_13 (Activation)   (None, 15, 64)            0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 7, 64)             0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 448)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               57472     
_________________________________________________________________
activation_14 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
__________

In [24]:
#cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn1d_1.fit(X_train, Y_train, batch_size=64, validation_data=(X_test1, Y_test1) ,epochs=10)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("20170827_mawilab_flow_000.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("20170827_mawilab_flow_000.h5")
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_Valaccuaracy00_01.png')

Train on 38361 samples, validate on 40780 samples
Epoch 1/10
38361/38361 [==============================] - 11s 278us/step - loss: 0.4811 - acc: 0.7044 - val_loss: 0.4744 - val_acc: 0.7358
Epoch 2/10
38361/38361 [==============================] - 9s 228us/step - loss: 0.4641 - acc: 0.7175 - val_loss: 0.4699 - val_acc: 0.7288
Epoch 3/10
38361/38361 [==============================] - 9s 228us/step - loss: 0.4600 - acc: 0.7198 - val_loss: 0.4679 - val_acc: 0.7373
Epoch 4/10
38361/38361 [==============================] - 9s 238us/step - loss: 0.4566 - acc: 0.7246 - val_loss: 0.4661 - val_acc: 0.7425
Epoch 5/10
38361/38361 [==============================] - 9s 235us/step - loss: 0.4540 - acc: 0.7254 - val_loss: 0.4610 - val_acc: 0.7407
Epoch 6/10
38361/38361 [==============================] - 9s 229us/step - loss: 0.4515 - acc: 0.7277 - val_loss: 0.4600 - val_acc: 0.7406
Epoch 7/10
38361/38361 [==============================] - 9s 230us/step - loss: 0.4503 - acc: 0.7290 - val_loss: 0.4623 -

In [25]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score
y_pred = cnn1d_1.predict(X_test1)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

y_pred = cnn1d_1.predict_classes(X_test1)
print(y_pred)

p=cnn1d_1.predict_proba(X_test1)

target_names = ['normal', 'anomaly']
print(classification_report(np.argmax(Y_test1, axis=1), y_pred, target_names=target_names, digits=4))
print(confusion_matrix(np.argmax(Y_test1, axis=1), y_pred))


print(f1_score(np.argmax(Y_test1, axis=1), y_pred, average='macro'))

print(f1_score(np.argmax(Y_test1, axis=1), y_pred, average='micro'))

print(f1_score(np.argmax(Y_test1, axis=1), y_pred, average='weighted'))

print(f1_score(np.argmax(Y_test1, axis=1), y_pred, average=None))

[[0.00307279 0.99702686]
 [0.06667804 0.9339693 ]
 [0.46960118 0.53060585]
 ...
 [0.51257503 0.48915505]
 [0.18943013 0.809674  ]
 [0.1998455  0.799925  ]]
[1 1 1 ... 0 1 1]
[1 1 1 ... 0 1 1]
             precision    recall  f1-score   support

     normal     0.5770    0.3397    0.4276     11774
    anomaly     0.7703    0.8989    0.8297     29006

avg / total     0.7145    0.7374    0.7136     40780

[[ 4000  7774]
 [ 2933 26073]]
0.6286487703732528
0.7374448258950466
0.7135839232184279
[0.4276474  0.82965014]
